# 1. Importing modules and functions

In [31]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Fingerprints import FingerprintMols
import chembl_structure_pipeline
import numpy as np
import pandas as pd
from rdkit.Chem import PandasTools
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import balanced_accuracy_score
import joblib
import pickle
from IPython.display import HTML
import matplotlib.pyplot as plt

# 2.Data entry and curation work set

In [2]:
uploaded_file_ws="datasets/HDAC6_ws.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetIntProp("Active"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  105 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [3]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [4]:
len(y_tr)

105

# 3.Standardization SDF file for work set

In [5]:
records_ws = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToMolBlock(all_mols_ws[i])
    records_ws.append(record)
            
mols_ws = []
for i,record in enumerate(records_ws):
    standard_record = chembl_structure_pipeline.standardize_molblock(record)
    m = Chem.MolFromMolBlock(standard_record)
    mols_ws.append(m)
           
moldf_ws = []
for val in mols_ws:
    if val != None:
        moldf_ws.append(val)
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  105 molecules


# 4.Data entry and curation test set

In [6]:
uploaded_file_ts="datasets/HDAC6_ts.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetIntProp("Active"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  26 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [7]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [8]:
len(y_ts)

26

# 5.Standardization SDF file for test set

In [9]:
records_ts = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToMolBlock(all_mols_ts[i])
    records_ts.append(record)
            
mols_ts = []
for i,record in enumerate(records_ts):
    standard_record = chembl_structure_pipeline.standardize_molblock(record)
    m = Chem.MolFromMolBlock(standard_record)
    mols_ts.append(m)
           
moldf_ts = []
for val in mols_ts:
    if val != None:
        moldf_ts.append(val)
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  26 molecules


# 6.Descriptor calculation for work set

In [10]:
fp_tr = [Chem.RDKFingerprint(m) for m in moldf_ws]

In [11]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [12]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [13]:
x_tr = pd.DataFrame(np.array(x_tr)) 

In [14]:
x_tr

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
102,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
103,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0


In [15]:
x_tr.to_csv('Models/Topological_fingerprints/x_tr.csv', index=True)

In [16]:
x_tr.shape

(105, 2048)

# 7.Descriptor calculation for test set

In [17]:
fp_ts = [Chem.RDKFingerprint(m) for m in moldf_ts]

In [18]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [19]:
x_ts = rdkit_numpy_convert(fp_ts)

In [20]:
x_ts = pd.DataFrame(np.array(x_ts)) 

In [21]:
x_ts

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
1,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
2,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
5,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
6,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
7,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
8,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
9,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
x_ts.to_csv('Models/Topological_fingerprints/x_ts.csv', index=True)

In [23]:
x_ts.shape

(26, 2048)

# 8.Neural network model building and validation 

## 8.1. Neural network model building 

In [57]:
parameters = {'activation': ['identity', 'logistic', 'tanh', 'relu'],'solver': ['lbfgs','sgd', 'adam'],'max_iter': [200, 800, 1000, 1400, 1800,2000], 'hidden_layer_sizes':[(500, 400, 300, 200, 100), (400, 400, 400, 400, 400), (300, 300, 300, 300, 300), (200, 200, 200, 200, 200), (100, 100, 100), (10, 10, 10), (500,), (100,), (10,)]}

In [58]:
mlp = GridSearchCV(MLPClassifier(), parameters)
mlp.fit(x_tr, y_tr)

C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  

C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  

C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1400) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1400) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1400) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1400) reached and the optimization hasn't converged yet

C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  

C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  

C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  

C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  

GridSearchCV(estimator=MLPClassifier(),
             param_grid={'activation': ['identity', 'logistic', 'tanh', 'relu'],
                         'hidden_layer_sizes': [(500, 400, 300, 200, 100),
                                                (400, 400, 400, 400, 400),
                                                (300, 300, 300, 300, 300),
                                                (200, 200, 200, 200, 200),
                                                (100, 100, 100), (10, 10, 10),
                                                (500,), (100,), (10,)],
                         'max_iter': [200, 800, 1000, 1400, 1800, 2000],
                         'solver': ['lbfgs', 'sgd', 'adam']})

In [59]:
mlp.best_params_

{'activation': 'logistic',
 'hidden_layer_sizes': (10, 10, 10),
 'max_iter': 1000,
 'solver': 'lbfgs'}

In [60]:
mlp = mlp.best_estimator_

## 8.2.  5-fold-cross-validation   Neural network model

In [27]:
seed = 42

In [159]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [233]:
y_pred_CV_mlp = cross_val_predict(mlp, x_tr, y_tr, cv=cv)

In [234]:
confusion_matrix_CV_mlp = metrics.confusion_matrix(y_tr, y_pred_CV_mlp, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_tr, y_pred_CV_mlp, weights='linear')
TN, FP, FN, TP = confusion_matrix_CV_mlp.ravel()
MCC=matthews_corrcoef(y_tr, y_pred_CV_mlp)
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))
print("MCC = ", round((MCC), 2))
print(TP)
print(TN)
print(FP)
print(FN)

balanced_accuracy =  0.81
SE =  0.76
SP =  0.87
Kappa =  0.63
MCC =  0.63
29
58
9
9


In [74]:
pickle.dump(mlp, open('Models/Topological_fingerprints/HDAC6_mlp_TFP_15_07_23.pkl', 'wb'))

## 8.3.Y-randomization for MLP model

In [183]:
permutations = 500
score, permutation_scores, pvalue = permutation_test_score(mlp, x_tr, y_tr,
                                                           cv=cv, scoring='balanced_accuracy',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 23.5min


True score =  0.81 
Y-randomization =  0.5 
p-value =  0.002


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 26.5min finished


In [262]:
max_Y_randomization = round(np.amax(permutation_scores, axis=0), 2) 
max_Y_randomization

0.67

In [263]:
standard_deviation = round(np.std(permutation_scores, axis=0), 3)
standard_deviation

0.054

In [264]:
min_Y_randomization = round(np.min(permutation_scores, axis=0), 2) 
min_Y_randomization

0.36

In [180]:
a = np.greater_equal(permutation_scores, score)
print("Coverage = ", sum(a) / len(a))

Coverage =  0.0


## 8.4. Model Neural network: predict for molecules of test set

In [132]:
y_pred_mlp = mlp.predict(x_ts)

In [133]:
y_pred_mlp

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0])

In [134]:
confusion_matrix_mlp = metrics.confusion_matrix(y_ts, y_pred_mlp, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_ts, y_pred_mlp, weights='linear')
TN, FP, FN, TP = confusion_matrix_mlp.ravel()
MCC=matthews_corrcoef(y_ts, y_pred_mlp)
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))
print("MCC = ", round((MCC), 2))
print(TP)
print(TN)
print(FP)
print(FN)

balanced_accuracy =  0.89
SE =  0.89
SP =  0.88
Kappa =  0.75
MCC =  0.75
8
15
2
1


# 9. Estimating applicability domain.

In [237]:
d_ECFP4 = {}
for mol in Chem.SDMolSupplier("datasets/HDAC6_ws.sdf"):
    tp = Chem.RDKFingerprint(mol)
    for m in Chem.SDMolSupplier('datasets/HDAC6_ts.sdf'):
        if m is not None:
            tp_ =  Chem.RDKFingerprint(m)
            d_ECFP4.setdefault(Chem.MolToSmiles(m),[]).append(DataStructs.FingerprintSimilarity(tp, tp_))

In [238]:
df_ECFP4 = pd.DataFrame.from_dict(d_ECFP4)

In [239]:
df_ECFP4.max()

O=C(CCCCCCC(=O)Nc1cccc(-c2cn(-c3ccc(I)cc3)nn2)c1)NO              0.952381
O=C(CCCCCCn1cc(-c2ccc(Nc3c4ccccc4nc4ccccc34)cc2)nn1)NO           0.991094
O=C(CCCCCCC(=O)Nc1cccc(-c2cn(C3CCCCC3)nn2)c1)NO                  0.598385
Cc1[nH]c2ccccc2c1CCNCc1ccc(C=CC(=O)NO)cc1                        0.540897
O=C(CCCCCCC(=O)Nc1ccc(-c2ccccc2)cc1)NO                           0.715719
Nc1ccc(-c2cc(C(=O)NCCCCCCC(=O)NO)no2)cc1                         0.967442
O=C(CCCCCCC(=O)Nc1cccc(-c2cn(Cc3ccc(O)cc3)nn2)c1)NO              0.557971
COc1cc2ccn(CCCCOc3cccc(NC(=O)CCCCCCC(=O)NO)c3)c2c(OC)c1OC        0.679037
COc1cc2ccn(CCOc3cccc(NC(=O)CCCCCCC(=O)NO)c3)c2c(OC)c1OC          0.746862
COc1ccc(Cl)cc1C(=O)NCCc1ccc(C=CC(=O)NO)cc1                       0.912308
CN(c1ccc(OCC(=O)NO)cc1)c1ncnc2ccccc12                            0.901515
COc1ccc(N(C)c2nc(C)nc3ccccc23)cc1OCCCC(=O)NO                     0.969388
CN(c1ccc(OCCCCCC(=O)NO)cc1)c1ncnc2ccccc12                        0.977865
COc1ccc2nc3cc(Cl)ccc3c(Nc3ccc(-c4cn(CC

In [240]:
threshold = 0.40
da_ECFP4 = np.asarray(df_ECFP4)
da_ECFP4 = np.amax(df_ECFP4, axis=0) >= threshold
da_ECFP4

O=C(CCCCCCC(=O)Nc1cccc(-c2cn(-c3ccc(I)cc3)nn2)c1)NO              True
O=C(CCCCCCn1cc(-c2ccc(Nc3c4ccccc4nc4ccccc34)cc2)nn1)NO           True
O=C(CCCCCCC(=O)Nc1cccc(-c2cn(C3CCCCC3)nn2)c1)NO                  True
Cc1[nH]c2ccccc2c1CCNCc1ccc(C=CC(=O)NO)cc1                        True
O=C(CCCCCCC(=O)Nc1ccc(-c2ccccc2)cc1)NO                           True
Nc1ccc(-c2cc(C(=O)NCCCCCCC(=O)NO)no2)cc1                         True
O=C(CCCCCCC(=O)Nc1cccc(-c2cn(Cc3ccc(O)cc3)nn2)c1)NO              True
COc1cc2ccn(CCCCOc3cccc(NC(=O)CCCCCCC(=O)NO)c3)c2c(OC)c1OC        True
COc1cc2ccn(CCOc3cccc(NC(=O)CCCCCCC(=O)NO)c3)c2c(OC)c1OC          True
COc1ccc(Cl)cc1C(=O)NCCc1ccc(C=CC(=O)NO)cc1                       True
CN(c1ccc(OCC(=O)NO)cc1)c1ncnc2ccccc12                            True
COc1ccc(N(C)c2nc(C)nc3ccccc23)cc1OCCCC(=O)NO                     True
CN(c1ccc(OCCCCCC(=O)NO)cc1)c1ncnc2ccccc12                        True
COc1ccc2nc3cc(Cl)ccc3c(Nc3ccc(-c4cn(CCCCCC(=O)NO)nn4)cc3)c2c1    True
CCN(CC)CCNC(=O)c1ccc

In [241]:
print("Coverage = ", sum(da_ECFP4) / len(da_ECFP4))

Coverage =  1.0


In [242]:
print(np.where(da_ECFP4 == 0)[0])

[]


In [243]:
out_Ad=list(np.where(da_ECFP4 == 0)[0])

In [244]:
out_Ad

[]

In [245]:
y_pred_mlp_ad=list(y_pred_mlp)

In [246]:
len(y_pred_mlp_ad)

26

In [247]:
y_pred_mlp_ad[:] = [x for i,x in enumerate(y_pred_mlp_ad) if i not in out_Ad]

In [248]:
len(y_pred_mlp_ad)

26

In [249]:
y_ts_ad=list(y_ts)
len(y_ts)

26

In [250]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts) if i not in out_Ad]
len(y_ts_ad)

26

In [251]:
confusion_matrix_ts = metrics.confusion_matrix(y_ts_ad, y_pred_mlp_ad, labels=[0,1])

In [252]:
Kappa = metrics.cohen_kappa_score(y_ts_ad, y_pred_mlp_ad, weights='linear')
TN, FP, FN, TP = confusion_matrix_ts.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
MCC=matthews_corrcoef(y_ts_ad, y_pred_mlp_ad)
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))
print("MCC = ", round((MCC), 2))
print(TP)
print(TN)
print(FP)
print(FN)

balanced_accuracy =  0.89
SE =  0.89
SP =  0.88
Kappa =  0.75
MCC =  0.75
8
15
2
1


# 10. Virtual Screening

## 10.1 Data entry and curation molecules for VS


In [67]:
uploaded_file_vs="datasets/HDAC6_vs_from_patents.sdf"
supplier_vs = Chem.ForwardSDMolSupplier(uploaded_file_vs,sanitize=False)
failed_mols_vs = []
all_mols_vs =[]
wrong_structure_vs=[]
wrong_smiles_vs=[]
y_vs = []
y_bad_index=[]
for i, m in enumerate(supplier_vs):
    structure = Chem.Mol(m)
    all_mols_vs.append(structure)
    y_vs.append(m.GetIntProp("HDAC6_class"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_vs.append(m)
        wrong_smiles_vs.append(Chem.MolToSmiles(m))
        wrong_structure_vs.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_vs), 'molecules')
print('Failed data: ', len(failed_mols_vs), 'molecules')
number_vs =[]
for i in range(len(failed_mols_vs)):
        number_ts.append(str(i+1))
bad_molecules_vs = pd.DataFrame({'No. failed molecule in original set': wrong_structure_vs, 'SMILES of wrong structure: ': wrong_smiles_vs, 'No.': number_vs}, index=None)
bad_molecules_vs = bad_molecules_vs.set_index('No.')
bad_molecules_vs

Original data:  12 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


In [68]:
y_vs[:] = [x for i,x in enumerate(y_vs) if i not in y_bad_index]
len(y_vs)

12

In [69]:
records_vs = []
for i in range(len(all_mols_vs)):
    record = Chem.MolToMolBlock(all_mols_vs[i])
    records_vs.append(record)
            
mols_vs = []
for i,record in enumerate(records_vs):
    standard_record = chembl_structure_pipeline.standardize_molblock(record)
    m = Chem.MolFromMolBlock(standard_record)
    mols_vs.append(m)
           
moldf_vs = []
for val in mols_vs:
    if val != None:
        moldf_vs.append(val)
print('Kept data: ', len(moldf_vs), 'molecules')

Kept data:  12 molecules


## 10.2 Descriptor calculation for VS

In [70]:
fp_vs = [Chem.RDKFingerprint(m) for m in moldf_vs]
def rdkit_numpy_convert(fp_vs):
    output = []
    for f in fp_vs:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [71]:
x_vs = rdkit_numpy_convert(fp_vs)

In [72]:
x_vs = pd.DataFrame(np.array(x_vs))
x_vs

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
6,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
7,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
8,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
9,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0


In [73]:
x_vs.shape

(12, 2048)

## 10.3 VS prediction

In [137]:
y_pred_mlp_vs = mlp.predict(x_vs)
y_pred_mlp_vs

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [138]:
confusion_matrix_mlp_vs = metrics.confusion_matrix(y_vs, y_pred_mlp_vs, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_vs, y_pred_mlp_vs, weights='linear')
TN, FP, FN, TP = confusion_matrix_mlp_vs.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
MCC=matthews_corrcoef(y_ts, y_pred_mlp_vs)
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))
print(TP)
print(TN)
print(FP)
print(FN)

balanced_accuracy =  0.78
SE =  0.89
SP =  0.67
Kappa =  0.56


## 10.4 Estimating applicability domain

In [139]:
d_TFP = {}
for mol in Chem.SDMolSupplier("datasets/HDAC6_ws.sdf"):
    tp = Chem.RDKFingerprint(mol)
    for m in Chem.SDMolSupplier('datasets/HDAC6_vs_from_patents.sdf'):
        if m is not None:
            tp_ =  Chem.RDKFingerprint(m)
            d_TFP.setdefault(Chem.MolToSmiles(m),[]).append(DataStructs.FingerprintSimilarity(tp, tp_))

In [140]:
df_TFP = pd.DataFrame.from_dict(d_TFP)

In [141]:
df_TFP.max()

COc1cc2ncn(OCCCCCC(=O)NO)c(=O)c2cc1OC                    0.429493
COc1cc2ncn(OCCCCCCC(=O)NO)c(=O)c2cc1OC                   0.432657
O=C(CCCCCOn1cnc2ccc(Br)cc2c1=O)NO                        0.347953
O=C(CCCCCCOn1cnc2ccc(Br)cc2c1=O)NO                       0.348430
COc1ccc(C2Nc3ccccc3C(=O)N2CCCCCC(=O)NO)cc1               0.429703
COc1cccc(C2Nc3ccccc3C(=O)N2CCCCCC(=O)NO)c1               0.415149
COc1cc(C2Nc3ccccc3C(=O)N2CCCCCC(=O)NO)cc(OC)c1OC         0.405788
O=C(CCCCCN1C(=O)c2ccccc2NC1c1ccc(Cl)cc1)NO               0.430140
COc1ccc(C2Nc3ccccc3C(=O)N2CCCCCC(=O)NO)cc1COc1ccccc1F    0.440127
COc1ccc(C2Nc3ccccc3C(=O)N2CCCCCCC(=O)NO)cc1              0.430129
O=C(CCCCCCN1C(=O)c2ccccc2NC1c1ccc(Cl)cc1)NO              0.429570
COc1ccc(C2Nc3ccc(F)cc3C(=O)N2CCCCCCC(=O)NO)cc1           0.407213
dtype: float64

In [142]:
threshold = 0.40
da_TFP = np.asarray(df_TFP)
da_TFP = np.amax(df_TFP, axis=0) >= threshold
da_TFP

COc1cc2ncn(OCCCCCC(=O)NO)c(=O)c2cc1OC                     True
COc1cc2ncn(OCCCCCCC(=O)NO)c(=O)c2cc1OC                    True
O=C(CCCCCOn1cnc2ccc(Br)cc2c1=O)NO                        False
O=C(CCCCCCOn1cnc2ccc(Br)cc2c1=O)NO                       False
COc1ccc(C2Nc3ccccc3C(=O)N2CCCCCC(=O)NO)cc1                True
COc1cccc(C2Nc3ccccc3C(=O)N2CCCCCC(=O)NO)c1                True
COc1cc(C2Nc3ccccc3C(=O)N2CCCCCC(=O)NO)cc(OC)c1OC          True
O=C(CCCCCN1C(=O)c2ccccc2NC1c1ccc(Cl)cc1)NO                True
COc1ccc(C2Nc3ccccc3C(=O)N2CCCCCC(=O)NO)cc1COc1ccccc1F     True
COc1ccc(C2Nc3ccccc3C(=O)N2CCCCCCC(=O)NO)cc1               True
O=C(CCCCCCN1C(=O)c2ccccc2NC1c1ccc(Cl)cc1)NO               True
COc1ccc(C2Nc3ccc(F)cc3C(=O)N2CCCCCCC(=O)NO)cc1            True
dtype: bool

In [143]:
print("Coverage = ", sum(da_TFP) / len(da_TFP))

Coverage =  0.8333333333333334


In [144]:
print(np.where(da_TFP == 0)[0])

[2 3]


## 10.5  Inside AD-only for  model

In [254]:
out_Ad=list(np.where(da_TFP == 0)[0])

In [255]:
out_Ad

[2, 3]

In [256]:
y_pred_mlp_ad=list(y_pred_mlp_vs)

In [257]:
y_pred_mlp_ad[:] = [x for i,x in enumerate(y_pred_mlp_ad) if i not in out_Ad]
len(y_pred_mlp_ad)

10

In [258]:
y_vs_ad=list(y_vs)
len(y_vs)

12

In [259]:
y_vs_ad[:] = [x for i,x in enumerate(y_vs) if i not in out_Ad]
len(y_vs_ad)

10

In [260]:
confusion_matrix_ts = metrics.confusion_matrix(y_vs_ad, y_pred_mlp_ad, labels=[0,1])

In [261]:
Kappa = metrics.cohen_kappa_score(y_vs_ad, y_pred_mlp_ad, weights='linear')
TN, FP, FN, TP = confusion_matrix_ts.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
MCC=matthews_corrcoef(y_vs_ad, y_pred_mlp_ad)
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))
print(TP)
print(TN)
print(FP)
print(FN)

balanced_accuracy =  0.83
SE =  1.0
SP =  0.67
Kappa =  0.74
7
2
1
0
